# Assignment nº1
## Lung Cancer Classification using Computerized Tomography (CT) Data
### Laboratory of Artificial Intelligence and Data Science (2024/25)

##### Work assembled by Alejandro Gonçalves (202205564), Francisca Mihalache (202206022), João Sousa (202205238) and Vítor Ferreira (201109428).

## Table of contents <a name="contents"></a>
1. [Introduction](#introduction)
2. [Business Understanding](#business)
8. [References](#references)

## Introduction <a name="introduction"></a>
[[go back to the top]](#contents)

The primary objective of this project is to develop a robust system for **predicting the malignancy** of lung nodules using radiomics, which could aid in the early diagnosis of lung cancer and minimize variability in human interpretation. 

To ensure a systematic and efficient approach, we will follow the **CRISP-DM** (Cross-Industry Standard Process for Data Mining) framework. CRISP-DM is a widely adopted methodology that consists of six phases: Business Understanding, Data Understanding, Data Preparation, Modeling, Evaluation, and Deployment. For the purposes of this academic project, our focus will be on the first five phases, covering all necessary aspects of model development, from problem definition to performance assessment, while excluding the deployment phase.

## Business Understanding <a name="business"></a>
[[go back to the top]](#contents)

Lung cancer remains the leading cause of cancer-related deaths globally, with survival rates highly dependent on the stage at which the disease is detected. Despite advancements in medical imaging, only about **16% of lung cancer cases** are diagnosed at an early, localized stage, when treatment is more likely to be effective **[1]**. For example, it is estimated that in 2024, approximately **234,580 new cases of lung cancer** will be diagnosed in the United States, with **125,070 deaths** resulting from the disease **[2]**. These statistics underscore the critical need for early detection, which can significantly improve patient outcomes, increasing the five-year survival rate from around 5% in advanced stages to over 50% in early stages.

**Computed Tomography (CT)** imaging has become a valuable, non-invasive tool for identifying lung nodules —potential indicators of lung cancer. However, traditional radiological analysis of CT images poses challenges: interpretations can vary among radiologists due to the subjective nature of image analysis, and reviewing large volumes of imaging data is time-consuming. To mitigate these limitations, **Computer-Aided Diagnosis (CAD) systems** have been developed to support radiologists by automatically evaluating the malignancy risk of lung nodules, potentially increasing diagnostic accuracy and efficiency.

A promising approach within CAD systems is **radiomics** — the extraction and analysis of a large number of quantitative features from medical images to provide non-invasive diagnostic and prognostic insights. Radiomics enables the transformation of imaging data into mineable, high-dimensional information, allowing for a more objective and detailed understanding of lung nodules. Moreover, the rise of machine learning (ML) and deep learning (DL) techniques, particularly Convolutional Neural Networks (CNNs), has demonstrated the potential to automatically learn and recognize patterns within medical images, facilitating improved classification and malignancy prediction.

Previous studies have utilized these techniques in conjunction with feature extraction tools to enhance classification performance. Despite their potential, however, these approaches face challenges in data privacy, standardization, and clinical validation. Addressing these issues is crucial to ensure reliable and ethically deployment in clinical settings, enabling improved early detection of lung cancer.

## References <a name="references"></a>
[[go back to the top]](#references)

**[1]** World Health Organization. "Latest Global Cancer Data: Cancer Burden Rises to 18.1 Million New Cases and 9.6 Million Cancer Deaths in 2018". Press Release Nº263; 2018.

**[2]** American Cancer Society. "Cancer Facts & Figures 2024". Atlanta: 2024.


